# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that the Retail Demo Store has trained models for 3 different use cases (Recommended for You, Customers who viewed X also viewed and Ranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with Recommenders and Campaigns and get recommendations. 

We will create and interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building  in this notebook highlighted in blue with a dashed outline.

![Workflow](images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [2]:
#retrieves previously stored variables 
%store -r 

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Adding some helper functions to make results more readable

First, let's create a supporting function to help make sense of the results returned by a Personalize recommender or campaign. Personalize returns only an `item_id`, and potentially information about whether that item was part of a promotion filter. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the synthetic dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: If you want to lookup information as part of the API request, consider implementing Real-Time Personalization APIs - https://github.com/aws-samples/personalization-apis.
</div>



In [4]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

In [5]:
def get_item_category_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['category'].values[0]
    return item_name

In [6]:
def get_item_style_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['style'].values[0]
    return item_name

Great! Now we have a way of rendering results. 

## Interact with Recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "Customers who viewed X also viewed" Recommender

"Customers who viewed X also viewed" requires an item and a user as input, and it will return customers also viewed based on an item that you specify.

The cells below will handle getting recommendations from the "Customers who viewed X also viewed" Recommender and rendering the results. Let's see what the recommendations are for an item.

We will be using the `recommenderArn`, the `itemId`, the `userId`, as well as the number or results we want, `numResults`.

#### Select a User

We'll just pick a random user for simplicity (there are about 6,000 users with user IDs assigned in sequential order). Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [7]:
user_id = 555
user_metadata_df[user_metadata_df['id']==user_id]

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
554,555,True,F,Natalie,Smith,natalie.smith@example.com,51,Natalie Smith,user555,outdoors_instruments_groceries,lower_priced_products,{},{'ios': {'anonymous_id': '6860933c-dce7-4ef0-b...,"[{'first_name': 'Natalie', 'last_name': 'Smith..."


We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [8]:
product_id = item_metadata_df.sample(1)["id"].values[0]

print ('productId: ', product_id)

productId:  0cb3ab29-b939-4732-b8ac-72ec61a4f950


First lets get the response directly from the get_recommendations API, which by default returns 25 items, but can be adjusted.

In [9]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_customers_who_viewed_arn,
    itemId = str(product_id),
    userId = str(user_id),
    numResults = 20
)

In [10]:
print (get_recommendations_response)

{'ResponseMetadata': {'RequestId': '29b9d5a0-5b92-4433-9570-4476f87f5d83', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 09 Mar 2024 22:49:43 GMT', 'content-type': 'application/json', 'content-length': '2456', 'connection': 'keep-alive', 'x-amzn-requestid': '29b9d5a0-5b92-4433-9570-4476f87f5d83'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '9bc87696-e9bd-4241-86b0-234e054a607b'}, {'itemId': '807e17f3-5daf-41c3-a07f-dbf6d48af7d9'}, {'itemId': '9b37c048-ae78-4095-9c27-5be95dbf5e3f'}, {'itemId': '40b1b91a-d1ff-40cf-8ea6-cea00103c8c7'}, {'itemId': '23aa70ab-959f-4835-a114-c30ff5e4f974'}, {'itemId': '83114893-a0bc-414c-b120-1c5ca8a35de4'}, {'itemId': '84dc124a-699d-433a-a828-5e32044c8945'}, {'itemId': 'aafaef40-a269-4d41-a813-e9bb95ce6aa7'}, {'itemId': 'e428be7e-1558-44d5-8f96-6a7374becfa7'}, {'itemId': '0ce88e06-0088-4591-8be1-329bb8241c9e'}, {'itemId': 'e5e842c6-9d8e-4940-a245-a0f02a5552ad'}, {'itemId': '62c19034-7048-43e8-a142-f6f8ba90e6a7'}, {'itemId': '8220a6f2-3e00-4ce9-af2d-

Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for products render in a nice dataframe. Again, let's create a helper function to achieve this.

In [11]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_viewed_x(recommendations_df, item_id, user_id):
    # Get the item name
    original_item_name = get_item_name_from_id(item_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_customers_who_viewed_arn,
        itemId = str(item_id),
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [original_item_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different items. Let's sample some data from our dataset to test our "Customers who viewed X also viewed" Recommender. Grab 5 random items from our dataframe.

In [12]:
samples = item_metadata_df.sample(5)
samples

,id,name,category,style,featured
2349,09b832ce-73fc-49f4-bd8b-5d5e4fe9fe72,Romantic Valentine''s Decorating Kit,seasonal,valentine,NaN
1079,7efdcf23-2d61-46df-9610-4a047da13a77,Cozy Gray Plush Sofa,furniture,sofas,NaN
1801,8c99de3f-95b9-4cc5-8e1a-87a5fac313fa,Resonant Grand Piano Inspires Musical Joy,instruments,keys,NaN
695,77a73995-ee61-4793-b48c-f9e1125fbce4,Vibrant Floral Bouquet Brightens Your Day,floral,centerpiece,NaN
2142,1664a77a-fcfc-46ee-8d4a-99be791248ff,Rugged Dog Hiking Pack,outdoors,pet,NaN


In [13]:
viewed_x_also_viewed_recommendations_df = pd.DataFrame()
items = samples.id.tolist()

for item in items:
    viewed_x_also_viewed_recommendations_df = get_new_recommendations_df_viewed_x(viewed_x_also_viewed_recommendations_df, item, user_id)

viewed_x_also_viewed_recommendations_df

,Romantic Valentine''s Decorating Kit,Cozy Gray Plush Sofa,Resonant Grand Piano Inspires Musical Joy,Vibrant Floral Bouquet Brightens Your Day,Rugged Dog Hiking Pack
0,Indulgent Valentine Truffle Treat,Sleek Dark Gray Contemporary Lounge Chair,Resonant Acoustic Drum Set,Vibrant Organic Floral Medley,"Succulent Salmon, Sustainably Farmed"
1,Sparkling Snowflake Christmas Ornament,Sleek Gray Workbench for Projects,Brass Magic in Your Hands,Green Fronds Graceful Houseplant,Smooth Sailing With This Reel
2,Sparkle the Season with Twinkling Lights,Mid-Century Tufted Armchair,Sleek Electric Guitar with Vibrant Tone,Vibrant Rose Bouquet,Adventure Awaits in this Sleek Kayak
3,Vintage Christmas Village Ornament,Sleek Dark Gray Worktable Boosts Productivity,Studio-Quality Microphone Captures Pristine Audio,Vibrant Air-Purifying Low-Maintenance Plant,Explore Nature with Your Pet
4,Bright Easter Decorations Liven Home,Minimalist White Dresser Brightens Bedrooms,"Vintage Acoustic Guitar, Warm Resonant Tones",Bright Lilies Floral Arrangement,Portable Camp Lamp - Light Your Adventures
5,Festive Snowflake Holiday Mug,Stylish Gray Sofa for Timeless Comfort,Vibrant Electric Guitar with Crisp Tone,Vibrant Tulip Bouquet,Fool Fish Every Time Lure
6,Spooky Wreath for Happy Haunting,Sturdy Pale Gray Work Table,"Sleek LED TV, Vivid Streaming",Vibrant Drought-Proof Houseplant,"Cast Far, Reel 'Em In"
7,Sleek Gainsboro Coffee Table Adds Minimalist S...,Sleek Glass Vase for Modern Style,Vibrant Acoustic Drum Kit,Spooky Halloween Decor Sign,"Paddle Faster, Longer with Sturdy Kayak"
8,Stylish Black Padded Dining Chair,Stylish Sienna Hiking Boots,Resonant Acoustic Guitar for Aspiring Musicians,Fragrant White Lily Arrangement,Sleek 88-Key Keyboard with Recording
9,Walnut Pie - Nutty Crunchy Delight,Stylish Pocket Clock Home Decor,Smooth Acoustic Bass for Deep Tones,Lush Green Air Purifying Indoor Plant,"Majestic Grand Piano, Powerfully Resonant"


Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the "Customers who viewed X also viewed" recommendations are based on the interactions we generated as input into the solution creation process above.

You may notice that some items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with a small dataset).

### "Recommended for you" Recommender

"Recommended for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

With this use case, Amazon Personalize automatically filters items the user purchased based on the `userId` that you specify and "Purchase" events. For better performance, include "Purchase" events along with the required "View" events. 

In [14]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20
)

In [15]:
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)


,Item,Name,Category,Style
0,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone",instruments,percussion
1,ccaec8f5-b33d-4676-a1b9-bc4b96951ee4,Garlic - Pungent Powerhouse for Health,groceries,vegetables
2,256e742a-d649-4833-a828-f3f69c26ab29,Zesty Hot Chillies Liven Any Dish,groceries,vegetables
3,bb3055a1-989f-4a8f-bf42-a010f19342e6,Sleek Electric Guitar for Expressive Playing,instruments,strings
4,3d11accd-9984-47e5-b0e6-94743e964b15,Durable Insulated Camp Cup,outdoors,camping
5,d2c48852-0c49-457c-bbf1-89de447ef4bb,"Paddle Faster, Longer with Sturdy Kayak",outdoors,kayaking
6,df051d82-4945-4570-a0b2-278487994b5b,Fresh-Caught Sustainable Jumbo Shrimp,groceries,seafood
7,29cf2176-3ba3-4762-b537-66bc226a7766,Gleaming Trumpet Sings Golden Tones,instruments,wind
8,d27b5683-8bcd-4c9d-a641-a8fe6b00f610,Fiery Fresh Chillies,groceries,vegetables
9,e1580149-87c5-4ff4-bb03-482465b67b16,Chewy Chocolate Chip Delight,groceries,bakery


Since "Recommended for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [16]:
samples = user_metadata_df.sample(3)
samples

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
2982,2983,True,F,Laura,Hughes,laura.hughes@example.com,42,Laura Hughes,user2983,footwear_jewelry_furniture,lower_priced_products,{},{'ios': {'anonymous_id': '2d341cf2-e98c-4faf-a...,"[{'first_name': 'Laura', 'last_name': 'Hughes'..."
3755,3756,True,F,Sharon,Palmer,sharon.palmer@example.com,25,Sharon Palmer,user3756,electronics_outdoors_footwear,lower_priced_products,{},{'ios': {'anonymous_id': '2123ded8-7f41-4243-8...,"[{'first_name': 'Sharon', 'last_name': 'Palmer..."
4091,4092,True,M,Keith,Wood,keith.wood@example.com,30,Keith Wood,user4092,housewares_tools_beauty,all_discounts,{},{'ios': {'anonymous_id': 'db5cc197-9c72-474d-9...,"[{'first_name': 'Keith', 'last_name': 'Wood', ..."


Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Recommended for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a product.

The cells below will handle getting recommendations from the "Recommended for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [17]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_recommended_for_you(recommendations_df, user_id):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [18]:
recommendations_df_users = pd.DataFrame()
users = samples.id.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_recommended_for_you(recommendations_df_users, user)

recommendations_df_users

,2983,3756,4092
0,Stylish Brown Leather Lace-Ups,Stylish Red Formal Shoes,Sleek Fast Boiling Electric Kettle
1,Stylish Saddle Brown Leather Boots,Stylish Sandals for Summer Fun,Fast Boil Kettle - Pour With Ease
2,Stylish Sandals for Summer Fun,Sleek Compact Camera Captures Life,Durable Spoon for Kitchen Mastery
3,Bold Neon Pink Formal Shoes,Sleek Black Heels Elevate Your Style,Ergonomic Screwdriver for Easy Driving
4,Hot Pink Pumps for Any Occasion,Capturing Memories in Focus,Versatile Pot - Stove to Oven
5,Bold Summer Sandals for Edgy Style,Stylish White Stiletto Heels,Stylish Floral Teapot
6,Stylish Square-Toe Sandals for Women,Capturing Memories in Stunning Clarity,Elevate Your Brew with This Elegant Beer Glass
7,Trendy Strappy Sandals for Stylish Comfort,"Sleek 4K Smart TV - Vivid Visuals, Intuitive I...",Quick-Boil Sleek Kettle
8,Stylish Black Stiletto Heels,Zoom In on Life's Memories,Stylish Ceramic Serving Bowl
9,Comfy Chic Sandals,Stylish Nude Heels,Handy Home Chef Tool Kit


## Applying the discount context

We'll get the user recommendations when discount context is applied for comparison. This is using the "contextual metadata" feature of Amazon Personalize. In this case, our context is whether a discount was applied.

In [19]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20,
    context = {'DISCOUNT':'Yes'} # Here we provide the context for the recommendations
)

item_list_context = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list_context ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list_context ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

,Item,Name,Category,Style
0,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone",instruments,percussion
1,ccaec8f5-b33d-4676-a1b9-bc4b96951ee4,Garlic - Pungent Powerhouse for Health,groceries,vegetables
2,256e742a-d649-4833-a828-f3f69c26ab29,Zesty Hot Chillies Liven Any Dish,groceries,vegetables
3,bb3055a1-989f-4a8f-bf42-a010f19342e6,Sleek Electric Guitar for Expressive Playing,instruments,strings
4,3d11accd-9984-47e5-b0e6-94743e964b15,Durable Insulated Camp Cup,outdoors,camping
5,d2c48852-0c49-457c-bbf1-89de447ef4bb,"Paddle Faster, Longer with Sturdy Kayak",outdoors,kayaking
6,df051d82-4945-4570-a0b2-278487994b5b,Fresh-Caught Sustainable Jumbo Shrimp,groceries,seafood
7,e1580149-87c5-4ff4-bb03-482465b67b16,Chewy Chocolate Chip Delight,groceries,bakery
8,d27b5683-8bcd-4c9d-a641-a8fe6b00f610,Fiery Fresh Chillies,groceries,vegetables
9,29cf2176-3ba3-4762-b537-66bc226a7766,Gleaming Trumpet Sings Golden Tones,instruments,wind


Let us compare it to the previous set of recommendations without context.

In [20]:
df = pd.DataFrame()
df['Item (No context)'] = [ itm['itemId'] for itm in item_list ]
df['Name (No context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]

df['Item (Context)'] = [ itm['itemId'] for itm in item_list_context ]
df['Name (Context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

,Item (No context),Name (No context),Item (Context),Name (Context)
0,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone",7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone"
1,ccaec8f5-b33d-4676-a1b9-bc4b96951ee4,Garlic - Pungent Powerhouse for Health,ccaec8f5-b33d-4676-a1b9-bc4b96951ee4,Garlic - Pungent Powerhouse for Health
2,256e742a-d649-4833-a828-f3f69c26ab29,Zesty Hot Chillies Liven Any Dish,256e742a-d649-4833-a828-f3f69c26ab29,Zesty Hot Chillies Liven Any Dish
3,bb3055a1-989f-4a8f-bf42-a010f19342e6,Sleek Electric Guitar for Expressive Playing,bb3055a1-989f-4a8f-bf42-a010f19342e6,Sleek Electric Guitar for Expressive Playing
4,3d11accd-9984-47e5-b0e6-94743e964b15,Durable Insulated Camp Cup,3d11accd-9984-47e5-b0e6-94743e964b15,Durable Insulated Camp Cup
5,d2c48852-0c49-457c-bbf1-89de447ef4bb,"Paddle Faster, Longer with Sturdy Kayak",d2c48852-0c49-457c-bbf1-89de447ef4bb,"Paddle Faster, Longer with Sturdy Kayak"
6,df051d82-4945-4570-a0b2-278487994b5b,Fresh-Caught Sustainable Jumbo Shrimp,df051d82-4945-4570-a0b2-278487994b5b,Fresh-Caught Sustainable Jumbo Shrimp
7,29cf2176-3ba3-4762-b537-66bc226a7766,Gleaming Trumpet Sings Golden Tones,e1580149-87c5-4ff4-bb03-482465b67b16,Chewy Chocolate Chip Delight
8,d27b5683-8bcd-4c9d-a641-a8fe6b00f610,Fiery Fresh Chillies,d27b5683-8bcd-4c9d-a641-a8fe6b00f610,Fiery Fresh Chillies
9,e1580149-87c5-4ff4-bb03-482465b67b16,Chewy Chocolate Chip Delight,29cf2176-3ba3-4762-b537-66bc226a7766,Gleaming Trumpet Sings Golden Tones


## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For an ECOMMERCE application you want to dynamically render a personalized shelf/rail/carousel based on some information (category, style, season, etc…). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within your system to generate the item list. You can use this campaign to rank the products listed for each category and the featured products list, as well as ranking catalog search results displayed in a search widget.

To demonstrate this, we will use the same user from before and a random collection of items.

#### Get Featured Products List

First let's get the list of featured products from the Products data.

In [21]:
featured_products = item_metadata_df[item_metadata_df['featured']==True]

#### Rank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rank for a specific user. This ranking allows us to provide ranked products based on the user's behavior. These behaviors should be consistent to the same persona that was mentioned above (since we're going to use the same `user_id`).

In [22]:
unranked_product_ids = list (featured_products['id'])

df = pd.DataFrame()
df['Item'] = [ itm for itm in unranked_product_ids ]
df['Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]
df['Category'] = [ get_item_category_from_id ( itm) for itm in unranked_product_ids ]
df['Style'] = [ get_item_style_from_id ( itm) for itm in unranked_product_ids ]
display (df)

,Item,Name,Category,Style
0,2b67230f-dc22-462e-9afe-c9e459f74093,Earthy Tan Handbag for Free Spirits,accessories,handbag
1,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,apparel,jacket
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,beauty,bathing
3,5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide,books,travel
4,3b145528-d5fc-4c2a-b2a5-e119128caa5f,"Block Outside Noise, Hear Pure Sound",electronics,headphones
5,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,floral,bouquet
6,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Bold Red Sneakers for All-Day Comfort,footwear,sneaker
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,furniture,sofas
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C,groceries,fruits
9,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,homedecor,decorative


Now let's have Personalize rank the featured product IDs based on our random user.

In [23]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

[
    {
        "itemId": "7160b264-e3ed-4ac3-9dd7-2c537b00e5ed",
        "score": 0.5511736
    },
    {
        "itemId": "3f9a39b2-0d63-4751-b6ee-4ecd08dd2276",
        "score": 0.3520552
    },
    {
        "itemId": "a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec",
        "score": 0.0573312
    },
    {
        "itemId": "5d37a44b-d121-426e-b528-59e603ba5923",
        "score": 0.0272063
    },
    {
        "itemId": "2b67230f-dc22-462e-9afe-c9e459f74093",
        "score": 0.0048989
    },
    {
        "itemId": "b87da3f8-9a3e-417d-abd7-16329c5be1ba",
        "score": 0.0030743
    },
    {
        "itemId": "8bffb5fb-624f-48a8-a99f-b8e9c64bbe29",
        "score": 0.0024234
    },
    {
        "itemId": "3b145528-d5fc-4c2a-b2a5-e119128caa5f",
        "score": 0.0006015
    },
    {
        "itemId": "6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b",
        "score": 0.0004217
    },
    {
        "itemId": "6f04daee-7387-442f-bc99-a9b0072b29ce",
        "score": 0.0003713
    },
    {
        "ite

We will add the ranked items as a second column to the original dataframe, for a side-by-side comparison.

In [24]:
df = pd.DataFrame()
df['Original List'] = [ itm for itm in unranked_product_ids]
df['Original List Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]

df['Reranking'] = [ itm['itemId'] for itm in reranked]
df['Reranking Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in reranked ]

display (df)

,Original List,Original List Name,Reranking,Reranking Name
0,2b67230f-dc22-462e-9afe-c9e459f74093,Earthy Tan Handbag for Free Spirits,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Fetch-tastic Frisbee for Playful Pups
1,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,3f9a39b2-0d63-4751-b6ee-4ecd08dd2276,Vibrant Tone Electric Guitar
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C
3,5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide,5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide
4,3b145528-d5fc-4c2a-b2a5-e119128caa5f,"Block Outside Noise, Hear Pure Sound",2b67230f-dc22-462e-9afe-c9e459f74093,Earthy Tan Handbag for Free Spirits
5,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin
6,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Bold Red Sneakers for All-Day Comfort,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Durable Screwdriver for Any Task
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,3b145528-d5fc-4c2a-b2a5-e119128caa5f,"Block Outside Noise, Hear Pure Sound"
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket
9,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Pumpkin Lights for Halloween


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user that cannot be easily categorized in your metadata, for instance "featured products" which are curated by a person.


# Filters <a class="anchor" id="filters"></a>

## Create Filters <a class="anchor" id="create-filters"></a>
[Back to top](#top)

Personalize can utilize either [static or dynamic filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Static filters are where the filter properties are built into the filter itself, which makes invocation simpler, but gives less flexibility. An example of this would be an accessories category filter, which invokes the get_recommendations_response api with the specific filter of CATEGORY_L1 = accessories. In order to create a recommendation for each filter, that would require 10+ filters. Personalize also supports dynamic filters, where the values can be passed at runtime, allowing for a single filter of CATEGORY_L1, where the actual category is passed at runtime. 

Filters can be created for fields of both Items and Events. 

A few common use cases for dynamic filters in ECOMMERCE are:

* Categorical filters based on Item Metadata - Often your item metadata will have information about the item. Filtering on these can provide recommendations within that data.

* Range based filters based on Item Metadata - Personalize supports range operations in both static and dynamic filters. Filtering based on a range can be used to create recommendations such as "Whats on sale now".

* Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "recommended for you" recommendation to a "purchase again" recommendations.



### Create Category Filter

Since there are a lot of categories to filter on, we will create a dynamic filter using the dynamic variable $CATEGORY, this will allow us to pass in the variable at runtime rather than create a static filter for each category.

In [25]:
category_filter_name = 'filter-category'

try:
    create_category_filter_response = personalize.create_filter(
        name = category_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.CATEGORY_L1 IN ($CATEGORY)'
    )
    
    category_filter_arn = create_category_filter_response['filterArn']
    print('Creating the Personalize filter with category_filter_arn {}.'.format(category_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    category_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+category_filter_name
    print('The Personalize filter {} already exists.'.format(category_filter_name))
    print ('\nWe will be using the existing Personalize Filter with category_filter_arn = {}'.format(category_filter_arn))
    

Creating the Personalize filter with category_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/filter-category.


### Create Purchased Products Filter

Since it's a poor user experience to recommend products that a user has already purchased, we will create a filter that excludes recently purchased products. We'll do this by creating a filter expression that excludes items that have an interaction with an event type of `Purchase` for the user. If you recall from the last notebook, the interactions dataset uses the `Purchase` event type to track purchased items.

In [26]:
filter_name = 'filter-purchased-products'

try:
    create_purchase_filter_response = personalize.create_filter(
        name=filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'EXCLUDE itemId WHERE INTERACTIONS.event_type in ("Purchase")'
    )
    
    purchase_filter_arn = create_purchase_filter_response['filterArn']

    print('Creating the Personalize filter with purchase_filter_arn {}.'.format(purchase_filter_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    purchase_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+filter_name
    print('The Personalize filter {} already exists.'.format(filter_name))
    print ('\nWe will be using the existing Personalize Filter with purchase_filter_arn = {}'.format(purchase_filter_arn))
    

Creating the Personalize filter with purchase_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/filter-purchased-products.


### Wait for Filter Status to Become ACTIVE

The filter should take a minute or so to become active.

In [27]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = purchase_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Purchase filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)
    
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = category_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Category Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

Purchase filter: CREATE PENDING
Purchase filter: CREATE IN_PROGRESS
Purchase filter: CREATE IN_PROGRESS
Purchase filter: CREATE IN_PROGRESS
Purchase filter: CREATE IN_PROGRESS
Purchase filter: CREATE IN_PROGRESS
Purchase filter: ACTIVE
Category Filter: ACTIVE


## Using Filters <a class="anchor" id="using-filters"></a>
[Back to top](#top)

Now that the Filters have been created we can use them to filter our recommendations.

## Category Filter<a class="anchor" id="filter">
[Back to top](#top)

To test our purchased products filter, we will request recommendations for a random user with a particular category. 
    
We will create a helper function to return recommendations using a dynamic flilter:

In [28]:
def get_new_recommendations_df_by_dynamic_filter(recommendations_df, user_id, category_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
        filterArn = category_filter_arn,
        filterValues = { "CATEGORY": "\"" + filter_values + "\""},
        numResults = 10
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        product = get_item_name_from_id(item['itemId'])
        recommendation_list.append(product)
    filter_name = category_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

We will select the first 7 categories for this example.

In [29]:
unique_category_field_values = list(item_metadata_df['category'].unique()[:7])
unique_category_field_values

['accessories',
 'apparel',
 'beauty',
 'books',
 'electronics',
 'floral',
 'footwear']

In [30]:
# Iterate through Categories
recommendations_df_category = pd.DataFrame()
for category in unique_category_field_values:
    recommendations_df_category = get_new_recommendations_df_by_dynamic_filter(recommendations_df_category, user_id, category_filter_arn , category)
    
recommendations_df_category

,accessories,apparel,beauty,books,electronics,floral,footwear
0,Stylish Canvas Tote Bag,Checkered Shirt for Any Occasion,Tranquil Bath Oil Escape,Italian Flavors Cookbook,"Sleek 4K Smart TV - Vivid Visuals, Intuitive I...",Vibrant Air-Purifying Low-Maintenance Plant,Stylish Sandals for Summer Fun
1,Sleek Dark Gray Travel Backpack,Performance Socks for Peak Athletic Moves,Scrub Away Dull Skin,Classic Swedish Cooking Recipes,Fast Desktop Computer for Work and Play,Breathtaking Locally-Grown Wedding Bouquet,Adventure in Pink Sneakers
2,Sleek Black Travel Backpack,Funky Retro Color Socks,Sharp Clipper for Smooth Trims,Wander Through Italy's Charm,Versatile Camera Captures Life's Moments,Season's Best Bouquet for Your Wedding,Stylish Nude Heels
3,Stylish Black Crossbody Bag,Sleek Black V-Neck Top,Indulgent Floral Soap for Pampering,Vivid Japan Travel Adventures,Immersive Sound Headphones,Breathtaking Wedding Bouquet of Flowers,Trendy Blue Sneakers for City Nights
4,Rugged Olive Backpack,Flattering Flowy Feminine Dress,Invigorating Citrus Aromatherapy Bar,Discover Russia's Hidden Gems Guidebook,Sleek Camera X100 - Professional Photos in You...,Crimson Cascade Rose Bouquet,Bold Summer Sandals for Edgy Style
5,Sleek Gray Travel Backpack,Playful Patterned Socks for Sassy Style,Refresh Skin with Hydrating Beauty Lotion,A Taste of Turkey,Sleek Laptop for Work and Play,Vibrant Floral Bouquet Brightens Your Day,Sleek Tan Leather Lace-Ups
6,Stylish Leather Handbag,Slim Light Gray Women's Versatile Shirt,Soft Lips with Shea,Spicy Thai Cuisine Cookbook,Sleek Connectivity Cable for Charging and Tran...,Locally-Grown Bridal Bouquet of Season's Best,Fun Flirty Sandals
7,Slate Pack - Roomy & Refined,Sparkling White Classic Shirt,Sleek Steel Scissors Snip with Precision,Spicy Flavors of Indonesian Cuisine,Film Precious Moments in HD,Pink and White Wedding Wreath,Comfy Chic Sandals
8,Stylish Accessory Belt for Women,Stylish Patterned Socks for Bold Flair,Plush Cotton Spa Towel,Spicy Thai Home Cooking Cookbook,Durable Cable Connects Your World,Vibrant Rose Bouquet,Sleek Brown Leather Boot Classics
9,Sleek Gray Pack for Jetsetting,Cozy Green Jacket for Outdoor Fun,Radiant Complexion Enhancer,Ancient Secrets of Mexico Travel Guide,Crisp Home Audio Speakers,Blush Blooms Bouquet for Wedded Bliss,Stylish Brown Leather Sneakers for Men


## Real-time Events<a class="anchor" id="event-tracking"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in retail workloads, because what products the user is interested in now may be different than what they were interested in a week ago, a day ago, or even a few minutes ago.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [31]:
event_tracker_name = 'personalize-poc-event-tracker'
try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    tracking_id = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(tracking_id))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    tracking_id = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd tracking_id = {}'.format(tracking_id))
    

{
  "eventTrackerArn": "arn:aws:personalize:us-east-1:381491864570:event-tracker/f22394de",
  "trackingId": "82b335cf-8b60-4c30-971a-d098ff86b70e",
  "ResponseMetadata": {
    "RequestId": "15b394a5-cac4-4a7b-9d2d-1280648c82a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 09 Mar 2024 22:51:36 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "139",
      "connection": "keep-alive",
      "x-amzn-requestid": "15b394a5-cac4-4a7b-9d2d-1280648c82a4"
    },
    "RetryAttempts": 0
  }
}

Creating the event_tracker with event_tracker_arn = arn:aws:personalize:us-east-1:381491864570:event-tracker/f22394de

And trackingId = 82b335cf-8b60-4c30-971a-d098ff86b70e


### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [32]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

EventTracker: CREATE PENDING
EventTracker: CREATE PENDING
EventTracker: ACTIVE


### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker. Since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. This challenge can be solved by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows applications to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, client applications can pass the provisional user ID to the user management system when creating a new user account. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

### Using the Purchased Products Filter

To use purchased products filter, we will request ranked recommendations for a random user. Then we will send a `Purchase` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter. The purchased product would be excluded from the new recommendations.

In [33]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = 456
display(user_metadata_df[user_metadata_df['id']==user_id])

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
455,456,True,M,Austin,Jensen,austin.jensen@example.com,35,Austin Jensen,user456,housewares_floral_seasonal,all_discounts,{},{'ios': {'anonymous_id': '1212fae7-a8eb-4709-b...,"[{'first_name': 'Austin', 'last_name': 'Jensen..."


In [34]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

,Item,Name,Category,Style
0,1e96e374-be23-4c97-b87e-b5c45cb8999f,Sleek Stainless Steel Pots and Pans,housewares,kitchen
1,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,floral,bouquet
2,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Durable Screwdriver for Any Task,tools,screwdriver
3,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,beauty,bathing
4,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Pumpkin Lights for Halloween,seasonal,halloween
5,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,furniture,sofas
6,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,apparel,jacket
7,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,homedecor,decorative
8,2ad09e8e-fd41-4d29-953e-546b924d7cb8,Sparkling Pendant Necklace,jewelry,necklace
9,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Fetch-tastic Frisbee for Playful Pups,outdoors,pet


Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [35]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')
print(f'Product name: {get_item_name_from_id ( product_id_to_purchase)}')

Product to simulate purchasing: 1e96e374-be23-4c97-b87e-b5c45cb8999f
Product name: Sleek Stainless Steel Pots and Pans


Next, let's send a `Purchase` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [36]:
response = personalize_events.put_events(
    trackingId = tracking_id,
    userId = str(user_id),
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'Purchase',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for Purchase event to become consistent.
time.sleep(10)

print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "bc869af8-fc74-4ca9-940f-bedeb5a9eef4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 09 Mar 2024 22:52:06 GMT",
      "content-type": "application/json",
      "content-length": "0",
      "connection": "keep-alive",
      "x-amzn-requestid": "bc869af8-fc74-4ca9-940f-bedeb5a9eef4"
    },
    "RetryAttempts": 0
  }
}


Finally, let's retrieve recommendations for the user again, but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.

In [37]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id),
    filterArn = purchase_filter_arn
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

,Item,Name,Category,Style
0,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,floral,bouquet
1,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Durable Screwdriver for Any Task,tools,screwdriver
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,beauty,bathing
3,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Pumpkin Lights for Halloween,seasonal,halloween
4,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,furniture,sofas
5,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,apparel,jacket
6,2ad09e8e-fd41-4d29-953e-546b924d7cb8,Sparkling Pendant Necklace,jewelry,necklace
7,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,homedecor,decorative
8,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Fetch-tastic Frisbee for Playful Pups,outdoors,pet
9,5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide,books,travel


The following code will raise an assertion error if the product we just purchased is still recommended.

In [38]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

Purchased item filtered from recommendations for user!


In [39]:
%store workshop_dataset_group_arn
%store region
%store role_name
%store s3_access_policy_arn

# Variables required for Optional Notebook 1 - Batch Inference
%store users
%store unranked_product_ids
%store bucket_name
%store workshop_rerank_solution_version_arn
%store item_metadata_df

Stored 'workshop_dataset_group_arn' (str)
Stored 'region' (str)
Stored 'role_name' (str)
Stored 's3_access_policy_arn' (str)
Stored 'users' (list)
Stored 'unranked_product_ids' (list)
Stored 'bucket_name' (str)
Stored 'workshop_rerank_solution_version_arn' (str)
Stored 'item_metadata_df' (DataFrame)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`04_Clean_Up.ipynb`](04_Clean_Up.ipynb).

You can choose to head to any of the optional notebooks in this folder to continue experimenting with Amazon Personalize.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: Run optional notebooks BEFORE you run the clean-up notebook.
</div>
